In [28]:
#%pip install kalshi-python
#%pip install kalshi

In [64]:

import kalshi_python

from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [65]:


demo_email = "imaksht@gmail.com" 
demo_password = "Demoakshatkalshi@1" 

demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"


exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [66]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'AAAGASD-23OCT12-US-3.663', 'event_ticker': 'AAAGASD-23OCT12-US', 'market_type': 'binary', 'title': 'Will average **gas prices** be above $3.663?', 'subtitle': '>$3.663', 'yes_sub_title': 'Rise', 'no_sub_title': 'Maintain or fall', 'open_time': '2023-10-11T14:00:00Z', 'close_time': '2023-10-12T13:55:00Z', 'expected_expiration_time': '2023-10-13T14:00:00Z', 'expiration_time': '2023-10-19T14:00:00Z', 'latest_expiration_time': '2023-10-19T14:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': 'Economics', 'risk_limit_cents': 250

In [67]:
nyc_indices = [ind for ind in range(len(markets_response['markets'])) if "high temp in NYC" in markets_response['markets'][ind]['title']]
nyc_indices
    

[25, 26, 27, 28, 29, 30]

In [68]:
    
# # Getting to next market objects from next page
# # Let's try it in action! Suppose we wanted to get the next 100 market objects...

# market_params = {'limit':100,
#                     'cursor':cursor, # passing in the cursor from the previous get_markets call
#                     'event_ticker': None,
#                     'series_ticker': None,
#                     'max_close_ts': None, # pass in unix_ts
#                     'min_close_ts': None, # pass in unix_ts
#                     'status': None,
#                     'tickers':None}

# markets_response = exchange_client.get_markets(**market_params)
# cursor = markets_response['cursor']

# print('keys:', markets_response.keys())
# print()
# print('number of objects:', len(markets_response['markets'])) # 100 objects!
# print()
# print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
# print()
# print('new cursor!', cursor)

In [69]:
# nyc_indices = [ind for ind in range(len(markets_response['markets'])) if "high temp in NYC" in markets_response['markets'][ind]['title']]


In [70]:
# nyc_indices

In [71]:

def place_order(ticker, side, count = 10, type = 'market'):

    order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':type,
                    'action':'buy',
                    'side':side,
                    'count':count,
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}
    print(order_params)

    exchange_client.create_order(**order_params)


In [72]:
def get_temps_steps(markets_response, nyc_indices):

    temp_steps = []
    ticker_list = []
    
    nyc_indices.sort()
    
    for ind in nyc_indices:
    
        l_tokens = markets_response['markets'][ind]['subtitle'].split()
    
        if "or" in markets_response['markets'][ind]['subtitle']:
    
            num = int(l_tokens[0][:-1])
    
            temp_steps.append(num)

            ticker_list.append(markets_response['markets'][ind]['ticker'])
    
        else:
    
            l_tokens = markets_response['markets'][ind]['subtitle'].split()
    
            num1 = int(l_tokens[0][:-1])
            num2 = int(l_tokens[-1][:-1])
    
            temp_steps.append([num1, num2])

            ticker_list.append(markets_response['markets'][ind]['ticker'])


    temp_steps.reverse()
    ticker_list.reverse()

    return temp_steps, ticker_list
    

In [73]:
temp_steps, ticker_list = get_temps_steps(markets_response, nyc_indices)

In [74]:
temp_steps

[62, [63, 64], [65, 66], [67, 68], [69, 70], 71]

In [75]:
def point_the_range(temp_steps, val):

    for i in range(len(temp_steps)):

        if type(temp_steps[i]) == list:

            if val <= temp_steps[i][1] and val >= temp_steps[i][0]:

                return i

            if i == 0 and val <= temp_steps[0][0]:
                return i

            if i == len(temp_steps)-1 and val >= temp_steps[-1][1]:
                return i
        else:

            if i == 0:

                if val <= temp_steps[0]:

                    return 0
            else:

                if val >= temp_steps[-1]:
                    return len(temp_steps)-1

    return -1
        

    
                    

In [76]:

import urllib.request

api_key = "FJMSWDF8QYLB5T5852LNJLGCK"


In [77]:
url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Central%20Park%20New%20York/2023-09-20/2023-10-08?unitGroup=us&maxDistance=1000&include=days&key={api_key}&elevationDifference=1&contentType=csv'
destination = '../Data/infer_temp.csv'

urllib.request.urlretrieve(url, destination)

('../Data/infer_temp.csv', <http.client.HTTPMessage at 0x16e0e14c0>)

In [78]:
import pandas as pd
import numpy as np
import pickle

In [79]:
file = open("../Model/lin_reg.pkl",'rb')
model = pickle.load(file)
file.close()

In [80]:
df = pd.read_csv(destination)


In [81]:
df.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [88]:
def give_next24hrs_pred(model, df):

    df.rename(columns = {'datetime':'DATE', 'tempmax':'TMAX', 'tempmin':'TMIN'}, inplace = True)

    df = df[['DATE', 'TMAX', 'TMIN']]

    df['TMAX_prev7'] = df['TMAX'].rolling(7).max()
    df['TMAX_prev3'] = df['TMAX'].rolling(3).max()
    df['TMAX_prev2'] = df['TMAX'].rolling(2).max()

    df['TMAX_prev1'] = df['TMAX'].rolling(1).max()

    df['TMIN_prev3'] = df['TMIN'].rolling(3).max()
    df['TMIN_prev2'] = df['TMIN'].rolling(2).max()
    df['TMIN_prev1'] = df['TMIN'].rolling(1).max()

    df = df.loc[7:, :]

    df['DATE'] = pd.to_datetime(df['DATE'])

    df['day'] = df['DATE'].dt.day
    df['month'] = df['DATE'].dt.month

    df.sort_values(by = ['DATE'], inplace = True)
    df.set_index("DATE", inplace = True)

    df.fillna(-1, inplace = True)

    df.drop(['TMIN', 'TMAX'], axis = 1, inplace = True)

    print(df)

    return model.predict(df)




In [83]:
df.columns
df.drop(['name']), axis = 1, inplace = True)

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [84]:
train_df, test_df = train_test_split(df)

In [85]:
test_df.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
16,"Central Park, New York, NY 10021, United States",2023-10-06,72.1,65.0,68.2,72.1,65.0,68.2,65.2,90.7,...,6.1,2,10,2023-10-06T06:57:21,2023-10-06T18:30:12,0.75,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,remote
12,"Central Park, New York, NY 10021, United States",2023-10-02,76.9,63.1,69.2,76.9,63.1,69.2,57.5,67.3,...,17.0,7,10,2023-10-02T06:53:14,2023-10-02T18:36:46,0.61,Clear,Clear conditions throughout the day.,clear-day,remote
17,"Central Park, New York, NY 10021, United States",2023-10-07,69.0,56.9,64.2,69.0,56.9,64.2,60.3,87.8,...,3.1,2,10,2023-10-07T06:58:23,2023-10-07T18:28:34,0.77,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,remote
8,"Central Park, New York, NY 10021, United States",2023-09-28,65.0,54.9,60.4,65.0,54.9,60.4,54.1,80.0,...,5.0,2,10,2023-09-28T06:49:11,2023-09-28T18:43:26,0.47,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,remote
9,"Central Park, New York, NY 10021, United States",2023-09-29,62.0,58.9,60.3,62.0,58.9,60.3,58.5,93.9,...,0.9,1,10,2023-09-29T06:50:11,2023-09-29T18:41:46,0.50,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,remote


In [90]:
X_train, X_test, y_train, y_test = train_df.drop(['tempmax'], axis = 1).values, test_df.drop(['tempmax'], axis = 1).values, train_df['tempmax'].values, test_df['tempmax'].values 

In [1]:
model = XGBRegressor()
model.fit(X_train, y_train)

In [2]:
preds = give_next24hrs_pred(model, df)

In [ ]:
pred = preds[-1]

In [ ]:
pred

60.68068

In [ ]:
our_pt = point_the_range(temp_steps, pred)

In [ ]:
max_ind = 1
max_dis =1
for ind in range(len(temp_steps)):
    if max_dis < abs(ind - our_pt):
        max_dis = abs(ind - our_pt)
        max_ind - ind
place_order(ticker_list[ind], 'no', count = 1)

{'ticker': 'HIGHNY-23OCT11-T70', 'client_order_id': 'a0a65db2-55d6-4f44-8969-63f7eb233dac', 'type': 'market', 'action': 'buy', 'side': 'no', 'count': 1, 'expiration_ts': None, 'sell_position_floor': None, 'buy_max_cost': None}
{'ticker': 'HIGHNY-23OCT11-T70', 'client_order_id': 'a0a65db2-55d6-4f44-8969-63f7eb233dac', 'side': 'no', 'action': 'buy', 'count': 1, 'type': 'market'}


In [ ]:
num_count = 100

for ind in range(len(temp_steps)):

    if ind != our_pt:

        try:

            count = int(abs(ind - our_pt) * num_count/len(temp_steps))
    
            place_order(ticker_list[ind], 'no', count = count)
        except:
            pass

{'ticker': 'HIGHNY-23OCT11-B63.5', 'client_order_id': '071c99a7-749c-4409-b3ae-5f877313c0c3', 'type': 'market', 'action': 'buy', 'side': 'no', 'count': 83, 'expiration_ts': None, 'sell_position_floor': None, 'buy_max_cost': None}
{'ticker': 'HIGHNY-23OCT11-B63.5', 'client_order_id': '071c99a7-749c-4409-b3ae-5f877313c0c3', 'side': 'no', 'action': 'buy', 'count': 83, 'type': 'market'}
{'ticker': 'HIGHNY-23OCT11-B65.5', 'client_order_id': 'eac2ed67-573e-4a38-bc7c-979fc085bc97', 'type': 'market', 'action': 'buy', 'side': 'no', 'count': 66, 'expiration_ts': None, 'sell_position_floor': None, 'buy_max_cost': None}
{'ticker': 'HIGHNY-23OCT11-B65.5', 'client_order_id': 'eac2ed67-573e-4a38-bc7c-979fc085bc97', 'side': 'no', 'action': 'buy', 'count': 66, 'type': 'market'}
{'ticker': 'HIGHNY-23OCT11-B67.5', 'client_order_id': '989febf8-60f3-4c5d-bc7e-6117ec1bc62c', 'type': 'market', 'action': 'buy', 'side': 'no', 'count': 50, 'expiration_ts': None, 'sell_position_floor': None, 'buy_max_cost': None

In [ ]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

{'balance': 1057279}

In [ ]:

ticker = 'HIGHNY-23OCT09-B58.5'

order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':'market',
                    'action':'buy',
                    'side':'no',
                    'count':10,
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}

exchange_client.create_order(**order_params)


# exchange_client.create_order(**order_params)             


{'ticker': 'HIGHNY-23OCT09-B58.5', 'client_order_id': 'f589b53e-60fb-4550-8129-6e88a124a093', 'side': 'no', 'action': 'buy', 'count': 10, 'type': 'market'}


{'order': {'order_id': '26c758bf-d959-4e59-b992-533b4f191dd5',
  'user_id': '29596cf5-9060-415b-b7bd-797be9b8f741',
  'ticker': 'HIGHNY-23OCT09-B58.5',
  'status': 'resting',
  'yes_price': 1,
  'no_price': 99,
  'created_time': '2023-10-09T18:48:15.393164Z',
  'expiration_time': None,
  'action': 'buy',
  'side': 'no',
  'type': 'market',
  'client_order_id': 'f589b53e-60fb-4550-8129-6e88a124a093',
  'order_group_id': ''}}

In [ ]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier